<a href="https://colab.research.google.com/github/hhamaguchi/BIG_DATA/blob/master/Projeto_final_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip3 install -q numpy scipy pandas matplotlib twython
!pip3 install -q numpy scipy pandas twython
!pip3 install --upgrade -q gspread

In [0]:
# Import dependencies
import pandas as pd
#import matplotlib.pyplot as plt
from twython import Twython
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [0]:
# Auth GDrive
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
# Credentials
APP_KEY = 'B6cZTTHoeMr96AFfOsbVM2lGG'
APP_SECRET = 'QhGt5p4TsGVK8tOds1fWc7hEDIFfrgnTW94fenpl1WneWLWqfz'
OAUTH_TOKEN = '167813147-tP4vOpAZFlcfRLlqXWMfRqcBW3yvjNpzRAborOUu'
OAUTH_TOKEN_SECRET = '8UshaZNMspgdN2kRuSlkPbBJfYdu3UDzOzZt0R1F8JoU7'

In [0]:
# Conn on twitter
twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

#twitter.verify_credentials()
#twitter.get_home_timeline()

In [0]:
# Query on twitter
def queryTwitterToDF(q):
  query = { 'q': q, 'count': 25, 'lang': 'pt', 'result_type': 'mixed'}
  result = twitter.search(**query)
  dict_ = {'user': [],'text': [], 'favorite_count': []}  
  for status in result['statuses']:  
      dict_['user'].append(status['user'])
      dict_['text'].append(status['text'])
      dict_['favorite_count'].append(status['favorite_count'])
  df = pd.DataFrame(dict_)  
  df.sort_values(by='favorite_count')
  return df





In [0]:
df_direita = pd.concat([queryTwitterToDF('#alckmin'), queryTwitterToDF('#extremaesquerda'),
                      queryTwitterToDF('#conservador'), queryTwitterToDF('#candidatodedireita'),
                      queryTwitterToDF('#bolsomito'), queryTwitterToDF('#partidodedireita'),
                      queryTwitterToDF('#PSDB'), queryTwitterToDF('#PFL'),
                      queryTwitterToDF('#petralhas')], ignore_index=True)
df_dir = df_direita.iloc[0:100,:]
#print(df_dir.shape)
#df_dir

In [0]:
df_esquerda = pd.concat([queryTwitterToDF('#lula'), queryTwitterToDF('#extremadireita'),
                      queryTwitterToDF('#lulalivre'), queryTwitterToDF('#candidatodeesquerda'),
                      queryTwitterToDF('#haddad'), queryTwitterToDF('#partidodeesquerda'),
                      queryTwitterToDF('#PT'), queryTwitterToDF('#golpe'),
                      queryTwitterToDF('#coxinhas')], ignore_index=True)
df_esq = df_esquerda.iloc[0:100,:]
#print(df_esq.shape)
#df_esq

In [0]:
df_neutro = pd.concat([queryTwitterToDF('-#lula'), queryTwitterToDF('-#extremadireita'),
                      queryTwitterToDF('-#lulalivre'), queryTwitterToDF('-#candidatodeesquerda'),
                      queryTwitterToDF('-#haddad'), queryTwitterToDF('-#partidodeesquerda'),
                      queryTwitterToDF('-#PT'), queryTwitterToDF('-#golpe'),
                      queryTwitterToDF('-#coxinhas'), queryTwitterToDF('-#alckmin'), 
                      queryTwitterToDF('-#extremaesquerda'), queryTwitterToDF('-#conservador'), 
                      queryTwitterToDF('-#candidatodedireita'), queryTwitterToDF('-#bolsomito'), 
                      queryTwitterToDF('-#partidodedireita'), queryTwitterToDF('-#PSDB'), queryTwitterToDF('-#PFL'),
                      queryTwitterToDF('-#petralhas')], ignore_index=True)

df_n = df_neutro.iloc[0:100,:]
#print(df_n.shape)
#df_n

In [0]:
#df_direita.columns

In [0]:
#df_esquerda['user']

In [0]:
sh = gc.open('twitter_politico')
direita_sheet = sh.get_worksheet(0)
esquerda_sheet = sh.get_worksheet(1)
neutro_sheet = sh.get_worksheet(2)

for index, row in df_dir.iterrows():
  direita_sheet.update_acell('A' + str(index + 1), row['text'])

for index, row in df_esq.iterrows():
  esquerda_sheet.update_acell('A' + str(index + 1), row['text'])

for index, row in df_n.iterrows():
  neutro_sheet.update_acell('A' + str(index + 1), row['text'])

In [13]:
# Load data to DF.
sh = gc.open('twitter_politico')
direita_sheet = sh.get_worksheet(0)
esquerda_sheet = sh.get_worksheet(1)
neutro_sheet = sh.get_worksheet(2)

# get_all_values gives a list of rows.
dict_ = {'text': [], 'label': []}
for row in direita_sheet.get_all_values():
  dict_['text'].append(row[0])
  dict_['label'].append(0)

for row in esquerda_sheet.get_all_values():
  dict_['text'].append(row[0])
  dict_['label'].append(1)

for row in neutro_sheet.get_all_values():
  dict_['text'].append(row[0])
  dict_['label'].append(2)
  
df = pd.DataFrame(dict_)

print(df.shape)


(300, 2)


In [14]:
# Creating TDIDF Matrix
count_vect = CountVectorizer()

X_TF = count_vect.fit_transform(df['text'])

tfidf_transformer = TfidfTransformer()
X = tfidf_transformer.fit_transform(X_TF)
print(type(X))
print(X.shape)
print(X)

<class 'scipy.sparse.csr.csr_matrix'>
(300, 1368)
  (0, 1298)	0.2255589986677397
  (0, 1134)	0.08901836891018995
  (0, 1074)	0.26611714586639
  (0, 987)	0.23144984057019644
  (0, 884)	0.2549567663175079
  (0, 852)	0.24583807226706486
  (0, 790)	0.26611714586639
  (0, 613)	0.08099893558050877
  (0, 599)	0.17446177617623862
  (0, 472)	0.2549567663175079
  (0, 447)	0.26611714586639
  (0, 414)	0.26611714586639
  (0, 413)	0.2549567663175079
  (0, 258)	0.26611714586639
  (0, 131)	0.1856221557251208
  (0, 90)	0.26611714586639
  (0, 79)	0.2549567663175079
  (0, 74)	0.26611714586639
  (1, 1352)	0.37078673451230126
  (1, 1258)	0.37078673451230126
  (1, 1225)	0.27155795315834713
  (1, 1167)	0.27155795315834713
  (1, 1163)	0.31429346317368545
  (1, 1082)	0.16425071490893572
  (1, 1013)	0.2388424353380627
  :	:
  (296, 235)	0.3074119421780736
  (296, 126)	0.3074119421780736
  (297, 1335)	0.6056490429815283
  (297, 869)	0.40521611358889265
  (297, 520)	0.6432996144225656
  (297, 333)	0.2348504718054

In [0]:
# Train ML
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_TF, df['label'])

In [0]:
# Predict
#docs_new = ['o lula deve ficar livre', 'politicos são corruptos', 'Marina ladrão', 'bom noite']
#X_new_counts = count_vect.transform(docs_new)
#X_new_tfidf = tfidf_transformer.transform(X_new_counts)

#predicted = clf.predict(X_new_tfidf)
#print(predicted)

#for doc, category in zip(docs_new, predicted):
#  if category == 0:
#    print('%r => %s' % (doc, "direita"))
#  elif category == 1:
#    print('%r => %s' % (doc, "esquerda"))
#  else:
#    print('%r => %s' % (doc, "neutro"))
      
    


In [0]:
#from tweepy import OAuthHandler
#from tweepy import API
#from tweepy import Cursor
#from datetime import datetime, date, time, timedelta
#from collections import Counter
#import sys


In [0]:
# Auth GDrive
#from google.colab import auth
#auth.authenticate_user()

#import gspread
#from oauth2client.client import GoogleCredentials

#gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
# Credentials
#APP_KEY = 'B6cZTTHoeMr96AFfOsbVM2lGG'
#APP_SECRET = 'QhGt5p4TsGVK8tOds1fWc7hEDIFfrgnTW94fenpl1WneWLWqfz'
#OAUTH_TOKEN = '167813147-tP4vOpAZFlcfRLlqXWMfRqcBW3yvjNpzRAborOUu'
#OAUTH_TOKEN_SECRET = '8UshaZNMspgdN2kRuSlkPbBJfYdu3UDzOzZt0R1F8JoU7'

In [18]:
import tweepy
from tweepy import OAuthHandler
from tweepy import API


#consumer_key='B6cZTTHoeMr96AFfOsbVM2lGG'
#consumer_secret='QhGt5p4TsGVK8tOds1fWc7hEDIFfrgnTW94fenpl1WneWLWqfz'
#access_token='167813147-tP4vOpAZFlcfRLlqXWMfRqcBW3yvjNpzRAborOUu'
#access_token_secret='8UshaZNMspgdN2kRuSlkPbBJfYdu3UDzOzZt0R1F8JoU7'

auth = tweepy.OAuthHandler(APP_KEY, APP_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
api = tweepy.API(auth)


number_of_tweets =10
tweets = api.user_timeline(screen_name = '@alanprando', count=10)

tmp = []

tweets_for_csv = [tweet.text for tweet in tweets]
for j in tweets_for_csv:
  tmp.append(j)

print(tmp)  

#for status in tweepy.Cursor(api.user_timeline, screen_name='@alanprando', tweet_mode='extended').items(10):
#  print(status.full_text)

  
#print(tmp.shape)

  
  

['RT @TheNoiteVideos: Acelera São Paulo https://t.co/z3waJJIIqA', 'RT @OGloboPolitica: Ex-assessor de @FlavioBolsonaro fez saques em 14 bairros, aponta relatório do Coaf https://t.co/R8UD8RIlyn https://t.co…', 'RT @joaoamoedonovo: Concordo com a avaliação de Paulo Guedes. O Sistema S deve ser revisto e suas contribuições, que atualmente são obrigat…', 'RT @joaoamoedonovo: Isso é NOVO.\n\n✅Redução de 21 para 11 secretarias\n✅Não morar em palácio\n✅Dispensar a frotas de de helicópteros e aviões…', 'RT @da_cia: Luciano Huck, meu, puta cara antenado com a Educação, bicho. Precisamos dessa galera https://t.co/twrT7bfKLJ', 'RT @gabrielazevedo: Uma informação básica: é possível se opor ao PT e ao PSL ao mesmo tempo? É possível. No Brasil, uma pessoa pode não gos…', 'RT @JovemPanNews: "Você quer ser preso?", diz Lewandowski a passageiro de avião em vídeo que circula nas redes sociais. https://t.co/teuzlb…', 'RT @janainalimasp: O NOVO assumiu o compromisso de acabar com os privilégios na políti

In [19]:
# Predict
docs_new_N= tmp
X_new_counts_N = count_vect.transform(docs_new_N)
X_new_tfidf_N = tfidf_transformer.transform(X_new_counts_N)

predicted_N = clf.predict(X_new_tfidf_N)
print(predicted_N)

for doc, category in zip(docs_new_N, predicted_N):
  if category == 0:
    print('%r => %s' % (doc, "direita"))
  elif category == 1:
    print('%r => %s' % (doc, "esquerda"))
  else:
    print('%r => %s' % (doc, "neutro"))


[0 1 0 1 0 1 1 1 1 1]
'RT @TheNoiteVideos: Acelera São Paulo https://t.co/z3waJJIIqA' => direita
'RT @OGloboPolitica: Ex-assessor de @FlavioBolsonaro fez saques em 14 bairros, aponta relatório do Coaf https://t.co/R8UD8RIlyn https://t.co…' => esquerda
'RT @joaoamoedonovo: Concordo com a avaliação de Paulo Guedes. O Sistema S deve ser revisto e suas contribuições, que atualmente são obrigat…' => direita
'RT @joaoamoedonovo: Isso é NOVO.\n\n✅Redução de 21 para 11 secretarias\n✅Não morar em palácio\n✅Dispensar a frotas de de helicópteros e aviões…' => esquerda
'RT @da_cia: Luciano Huck, meu, puta cara antenado com a Educação, bicho. Precisamos dessa galera https://t.co/twrT7bfKLJ' => direita
'RT @gabrielazevedo: Uma informação básica: é possível se opor ao PT e ao PSL ao mesmo tempo? É possível. No Brasil, uma pessoa pode não gos…' => esquerda
'RT @JovemPanNews: "Você quer ser preso?", diz Lewandowski a passageiro de avião em vídeo que circula nas redes sociais. https://t.co/teuzlb…' => e

In [20]:
#import pandas as pd
import numpy as np
import scipy.stats

def descriptive_stats(distribuition):

  dist = np.array(distribuition)
  
  print 'scores', len(dist)

#d = {'name':pd.Series(['1','2','3','4','5','6','7','8','9','10']), 'Categoria':pd.Series(predicted_N.T)}
#d = {'Categoria':pd.Series(predicted_N.T)}

#df = pd.DataFrame(d)
#print(df.shape)
#print df. describe(include=['all'])


#results = predicted_N.count()
#print(results)

#from scipy import stats
#for arr in predicted_N.T:
#  print(stats.describe(arr))
  
#print("valor medio", predicted_N.mean()) 
#print("valor maximo", predicted_N.max()) 
#print("valor minimo", predicted_N.min()) 
#print("valor minimo", predicted_N.shape()) 
descriptive_stats(predicted_N)

  
  

SyntaxError: ignored